In [28]:
import json
import os
import numpy as np
import pandas as pd

emotion_label_policy = {'angry': 0, 'anger': 0,
    'disgust': 1,
    'fear': 2,
    'happy': 3, 'happines': 3, 'happiness': 3, 'excited': 3,
    'sad': 4, 'sadness': 4, 'frustrated': 4,
    'surprise': 5, 'surprised': 5, 
    'neutral': 6}

In [27]:
# json 데이터셋 파일 열기
datasets = ['../dataset/dailydialog_train.json', '../dataset/dailydialog_valid.json', '../dataset/dailydialog_test.json']

# 각 데이터셋의 데이터를 하나의 리스트로 합치기
with open(datasets[0], 'r', encoding='utf-8') as f:
    train = json.load(f)
with open(datasets[1], 'r', encoding='utf-8') as f:
    valid = json.load(f)
with open(datasets[2], 'r', encoding='utf-8') as f:
    test = json.load(f)

# concate 3 dictionaries 
valid.update(test)
train.update(valid)
total_data = train

# dataset 폴더에 'dailydialog_total.json' 파일이 없으면, total_data를 json 파일로 저장
if not os.path.isfile('../dataset/dailydialog_total.json'):
    with open('../dataset/dailydialog_total.json', 'w', encoding='utf-8') as f:
        json.dump(total_data, f, ensure_ascii=False, indent='\t')
        
dialog_ids = list(total_data.keys())

In [38]:
total_data['tr_10644'][0]

[{'turn': 1,
  'speaker': 'A',
  'utterance': 'Is there a problem , officer ?',
  'emotion': 'neutral'},
 {'turn': 2,
  'speaker': 'B',
  'utterance': 'I stopped you for driving through a red light .',
  'emotion': 'neutral'},
 {'turn': 3,
  'speaker': 'A',
  'utterance': 'Did I really ?',
  'emotion': 'surprise',
  'expanded emotion cause evidence': [2],
  'expanded emotion cause span': ['stopped you for driving through a red light .'],
  'type': ['inter-personal']},
 {'turn': 4,
  'speaker': 'B',
  'utterance': "You didn't know ?",
  'emotion': 'surprise',
  'expanded emotion cause evidence': [3],
  'expanded emotion cause span': ['Did I really ?'],
  'type': ['inter-personal']},
 {'turn': 5,
  'speaker': 'A',
  'utterance': 'I had no idea that I ran it .',
  'emotion': 'neutral'},
 {'turn': 6,
  'speaker': 'B',
  'utterance': "Don't you know that yellow means slow down ?",
  'emotion': 'neutral'},
 {'turn': 7,
  'speaker': 'A',
  'utterance': 'Yes , madam .',
  'emotion': 'neutral'}

In [45]:
# table1_columns = ['dialog_id', 'emotion', 'num_utterances']
table_columns = ['dialog_id', 'uid', 'speaker', 'utterance', 'emotion', 'cause turn', 'cause span']

# 각 dialog가 들어갈 table 선언하고 내용물 채우기 (column은 table_columns)
# key는 dialog_id와 uid의 조합
# value는 speaker, utterance, emotion, cause turn, cause span
# table: dialog_id, uid, speaker, utterance, emotion, cause turn, cause span

# 각 row를 저장할 데이터프레임
table = pd.DataFrame(columns=table_columns)
for dialog_id in total_data.keys():
    dialog = total_data[dialog_id][0]
    for utterance in dialog:
        turn = utterance['turn']
        uid = f'{dialog_id}_{turn}'
        if utterance['emotion'] != 'neutral':
            row = pd.DataFrame([[dialog_id, turn, utterance['speaker'], utterance['utterance'], utterance['emotion'], utterance['expanded emotion cause evidence'], utterance['expanded emotion cause span']]], columns=table_columns)
        else:
            row = pd.DataFrame([[dialog_id, turn, utterance['speaker'], utterance['utterance'], utterance['emotion'], '', '']], columns=table_columns)
        table = pd.concat([table, row], ignore_index=True)
        
        
    # table1: dialog_id, emotion, num_utterances
    
    

In [46]:
table

,dialog_id,uid,speaker,utterance,emotion,cause turn,cause span
0,tr_4466,1,A,"Hey , you wanna see a movie tomorrow ?",happiness,[1],[see a movie tomorrow ?]
1,tr_4466,2,B,Sounds like a good plan . What do you want to ...,happiness,[1],[see a movie tomorrow ?]
2,tr_4466,3,A,How about Legally Blonde .,neutral,,
3,tr_4466,4,B,"Ah , my girlfriend wanted to see that movie . ...",neutral,,
4,tr_4466,5,A,Isn't that a scary movie ?,neutral,,
...,...,...,...,...,...,...,...
11099,tr_10060,6,B,"Yes , of course .",happiness,[5],[Can you give her a message]
11100,tr_10060,7,A,Please tell her to give Colin a call when she ...,neutral,,
11101,tr_10060,8,B,Sure .,happiness,"[5, 7]","[Can you give her a message, Please tell her t..."
11102,tr_10060,9,A,Thank you for your help .,happiness,"[5, 7, 8]","[Can you give her a message, Please tell her t..."


In [ ]:
table1_columns = ['dialog_id', 'emotion', 'num_utterances']
table2_columns = ['utterance_id', 'dialog_id', 'speaker', 'utterance', 'emotion']